In [1]:
import json
import pandas as pd
import numpy as np
import glob
import os

In [2]:
PIXELS_X = 910
PIXELS_Y = 910  # equal to the number of scan lines

In [3]:
predictions_file = '/Users/darylwilding-mcbride/Downloads/experiments/dwm-test/predictions/yolov3-tiny_3l-tile-34-predictions.json'

In [4]:
with open(predictions_file) as file:
    predictions = json.load(file)

In [11]:
server_url = 'http://spectra-server-lb-1653892276.ap-southeast-2.elb.amazonaws.com/tile/{}/frame/{}'

In [14]:
# for each frame, generate the Via annotations
annotations = {}
for frame in predictions:
    tile_name = os.path.basename(frame['filename'])
    frame_id = int(f.split('-')[1])
    tile_id = int(f.split('-')[3])
    
    entry_name = 'tile-{}-frame-{}'.format(tile_id, frame_id)
    entry = {}
    entry['file_attributes'] = {}
    entry['filename'] = server_url.format(tile_id, frame_id)
    regions = []
    
    for idx, prediction in enumerate(frame['objects']):
        class_id = prediction['class_id']
        class_name = prediction['name']
        bounding_box = prediction['relative_coordinates']
        confidence = prediction['confidence']
        
        # calculate the coordinates
        x1 = (bounding_box['center_x'] - (bounding_box['width'] / 2)) * PIXELS_X
        y1 = (bounding_box['center_y'] - (bounding_box['height'] / 2)) * PIXELS_Y
        w = bounding_box['width'] * PIXELS_X
        h = bounding_box['height'] * PIXELS_Y
        x2 = x1 + w
        y2 = y1 + h
        
        # create the region
        region_attributes = {'charge':'{}+'.format(class_id+2), 'isotopes':'0'}
        shape_attributes = {'name':'rect', 'width': int(w), 'height': int(h), 'x': int(x1), 'y': int(y1)}
        region = {'region_attributes':region_attributes, 'shape_attributes':shape_attributes}

        # add this prediction to the region list
        regions.append(region)

    entry['regions'] = regions
    entry['size'] = -1
    
    annotations[entry_name] = entry


In [16]:
annotations_filename = '/Users/darylwilding-mcbride/Downloads/annotations-from-predictions-tile-34.json'

In [17]:
with open(annotations_filename, 'w') as file:
    json.dump(annotations, file)